In [2]:
# core importations
import caveclient
from caveclient import CAVEclient
import cloudvolume
from cloudvolume import CloudVolume
import pandas as pd
import numpy as np

# dash importations
import dash
from dash import Dash, dcc, html, Input, Output, State, dash_table

In [21]:
%%time
# ----------CORE DATA GATHERING FUNCTIONS-----------#

# defines function to convert nm coordinates into FlyWire-usable
def coordConvert(coords):
    x = coords
    x[0] /= 4
    x[1] /= 4
    x[2] /= 40
    return x

# defines function to convert list of coordinates in [4,4,40] resolution to root id #
def coordsToRoot(coords):
    # sets client #
    client = CAVEclient("flywire_fafb_production")

    # sets cloud volume #
    cv = cloudvolume.CloudVolume("graphene://https://prod.flywire-daf.com/segmentation/1.0/fly_v31", use_https=True)

    # determines resolution of volume #
    res = cv.resolution

    # converts coordinates using volume resolution #
    cv_xyz = [int(coords[0]/(res[0]/4)),int(coords[1]/(res[1]/4)),int(coords[2]/(res[2]/40))]

    # sets point by passing converted coordinates into 'download_point' method #
    point = int(cv.download_point(cv_xyz, size=1))

    # looks up root id for that supervoxel using chunkedgraph #
    root_result = client.chunkedgraph.get_root_id(supervoxel_id=point)

    return root_result

# defines function to get list of root ids from list of nucleus ids
def getRootNuc(nucleus_ids):
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    #determines current materialization version#
    mat_vers = max(client.materialize.get_versions())
    #makes nuc df using root ids#
    nuc_df = client.materialize.query_table('nuclei_v1',
                                            filter_in_dict={"id": nucleus_ids},
                                            materialization_version = mat_vers)
    #makes ouput list of root and nuc id lists aligned after possible reordering during query#
    output_list = [nuc_df['id'].tolist(), nuc_df['pt_root_id'].tolist()]
    return root_list

# defines function to get synapse info using root ID#
def getSyn(root_id):
    #sets client#
    client = CAVEclient("flywire_fafb_production")
    
    #gets current materialization version#
    mat_vers = max(client.materialize.get_versions())
    
    #makes df of presynapses (outputs)#
    pre_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"pre_pt_root_id":[root_id]})
    
    #makes df of postsynapses (inputs)#
    post_syn_df = client.materialize.query_table('synapses_nt_v1', 
                                            filter_in_dict={"post_pt_root_id":[root_id]})
    
    #calculates total pre and post synapses#
    pre_count = len(pre_syn_df)
    post_count = len(post_syn_df)
    
    #calculates neurotransmitter averages#
    pre_gaba_mean = pre_syn_df['gaba'].mean()
    pre_ach_mean = pre_syn_df['ach'].mean()
    pre_glut_mean = pre_syn_df['glut'].mean()
    pre_oct_mean = pre_syn_df['oct'].mean()
    pre_ser_mean = pre_syn_df['ser'].mean()
    pre_da_mean = pre_syn_df['da'].mean()
    post_gaba_mean = post_syn_df['gaba'].mean()
    post_ach_mean = post_syn_df['ach'].mean()
    post_glut_mean = post_syn_df['glut'].mean()
    post_oct_mean = post_syn_df['oct'].mean()
    post_ser_mean = post_syn_df['ser'].mean()
    post_da_mean = post_syn_df['da'].mean()
    
    #gets lists of pre and post synaptic partners#
    downstream_partners = pre_syn_df['post_pt_root_id'].unique()
    upstream_partners = post_syn_df['pre_pt_root_id'].unique()

    #makes blank output dataframe#
    out_df = pd.DataFrame()
    
    #assigns columns of output dataframe#
    out_df['Root ID'] = [root_id]
    out_df['Post Count'] = [post_count]
    out_df['Post Gaba Avg'] = [post_gaba_mean]
    out_df['Post Ach Avg'] = [post_ach_mean]
    out_df['Post Glut Avg'] = [post_glut_mean]
    out_df['Post Oct Avg'] = [post_oct_mean]
    out_df['Post Ser Avg'] = [post_ser_mean]
    out_df['Post Da Avg'] = [post_da_mean]
    out_df['Upstream Partners'] = [upstream_partners]
    out_df['Pre Count'] = [pre_count]
    out_df['Pre Gaba Avg'] = [pre_gaba_mean]
    out_df['Pre Ach Avg'] = [pre_ach_mean]
    out_df['Pre Glut Avg'] = [pre_glut_mean]
    out_df['Pre Oct Avg'] = [pre_oct_mean]
    out_df['Pre Ser Avg'] = [pre_ser_mean]
    out_df['Pre Da Avg'] = [pre_da_mean]
    out_df['Downstream Partners'] = [downstream_partners]
    
    return out_df

# defines function to build dataframe using root id list and 'options' (list of keywords based on checkboxes)
def dfBuilder(id_list,input_type,options):

    if input_type == 'root':
        
        for i in id_list:
            df_row = getSyn(i)
            out_df
    elif input_type == 'nuc':
    elif input_type == 'coord':
    else:
        #ADD ERROR MESSAGE HERE?#
    
    
    
    root_list = [str(i) for i in root_list]             #ensures root ids are formatted as strings to avoid rounding#
    df = pd.DataFrame(root_list, columns = ['Root ID']) #creates dataframe out of root_id list#

    if 'nucs' in options:
        nuc_df = getNucRoot(root_list)
        df['Nucleus ID'] = nuc_df['id']
        df['Nucleus Coordinates'] = nuc_df['pt_position'] #adds nucleus coordinates to df#
#         df['Nucleus Volume'] = nuc_df['volume'] #TEMPORARILY REMOVED#

    if 'syns' in options:
        pre_list = []
        post_list = []
        total_list = []
        for i in root_list:
            try:
                pre_list.append(getPreSyn(i))      #pulls synapse data#
            except:
                pre_list += ['n/a']
            try:
                post_list.append(getPostSyn(i))
            except:
                post_list += ['n/a']
            try:
                total_list.append(int(pre_list[-1] + post_list[-1]))
            except:
                total_list += ['n/a']
                
        df['Incoming Synapses'] = post_list
        df['Outgoing Synapses'] = pre_list
        df['Total Synapses'] = total_list
    
    if 'edits' in options: #CHANGE THIS TO TABULAR LOG#
        split_list = []
        merge_list = []
        edit_list = []
        for i in root_list:
            try:
                change_dict = getEdits(i)                         #pulls changelog#
                split_list.append(change_dict['n_splits'])        #appends split list#
                merge_list.append(change_dict['n_mergers'])       #appends merge list#
                edit_list.append(split_list[-1] + merge_list[-1]) #appends edit list#
            except:
                split_list += ['n/a']
                merge_list += ['n/a']
                edit_list += ['n/a']
        df['Splits'] = split_list
        df['Mergers'] = merge_list
        df['Edits'] = edit_list
           
    return df

Wall time: 0 ns


In [ ]:
# DASH APP #

app = dash.Dash(__name__)

# defines layout of various app elements (submission field, checkboxes, button, output table)#
app.layout = html.Div([
    dcc.Textarea(
        id='message_text',
        value='Choose lookup method from dropdown, input coordinates, select output parameters, and click "Submit" button.\nID queries are limited to 20 entries, coordinate lookups must be done one at a time.\nLookup takes ~2 seconds per entry.',
        style={'width': '800px','resize': 'none'},
        rows=3,
        disabled=True,
    ),
    html.Div(dcc.Input(  #defines input field#
        id='input_field', 
        type='text', 
        placeholder='ID Number',
    )),
    html.Br(
    ),
    dcc.Dropdown(
        id='query_type',
        options=[
            {'label': 'Root ID', 'value': 'root_query'},
            {'label': 'Nucleus ID', 'value': 'nuc_query'},
            {'label': 'Coordinates (batch coordinate input not currently supported)', 'value': 'coord_query'},
        ],
        value='root_query',
        style={'max-width': '500px'},
    ),
    html.Br(
    ),
    html.Div(dcc.Checklist(  #defines data selection checkboxes#
        id='checkboxes',
        options=[
            {'label': 'Nucleus Info', 'value': 'nucs'},
            {'label': 'Synapse Count', 'value': 'syns'},
            {'label': 'Edits', 'value': 'edits'},
        ],
        labelStyle={'display': 'block'},
        value=['nucs','syns','edits'],
    )),
    html.Br(
    ),
    html.Button(  #defines submission button#
        'Submit', 
        id='submit_button', 
        n_clicks=0,
    ),
    html.Br(
    ),
    html.Div(dash_table.DataTable(  #defines output table#
        id='table', 
        fill_width=False,  #sets column width to fit text instead of expanding to container width#
        export_format="csv",
    ))
])

# defines callback that takes root ids and desired data selection on button click and generates table
@app.callback(
    Output('table','columns'),           #defines first output location as the 'columns' aspect of 'table'#
    Output('table', 'data'),             #defines second output location as the 'data' aspect of 'table'#
    Output('message_text','value'),      #defines second output location as the 'data' aspect of 'table'#
    Input('submit_button', 'n_clicks'),  #defines trigger as button press (change in the state of the 'n_clicks' aspect of 'submit_button')# 
    State('query_type', 'value'),        #defines first input state as value of 'query_type'#
    State('input_field', 'value'),       #defines second input state as the value of 'input_field'#
    State('checkboxes','value'),         #defines third input state as the value of 'checkboxes'#
    prevent_initial_call=True,
)
def update_output(n_clicks, query_method, ids, checked):
    id_list = str(ids).split(",")                             #splits 'roots' string into list#
    id_list = [int(x.strip(' ')) for x in id_list]            #strips spaces from root_list entries and converts to integers#
    if len(id_list) <= 20:
        if query_method == 'root_query':                          #creates dataframe for root id queries#
            df = dfBuilderRoot(id_list, checked)
        elif query_method == 'nuc_query':                         #creates dataframe for nucleus id queries#
            df = dfBuilderNuc(id_list, checked)
        elif query_method == 'coord_query':                       #creates dataframe for coordinate query#
            derived_id = [coordsToRoot(id_list)]
            df = dfBuilderRoot(derived_id, checked)
        column_list = [{"name": i, "id": i} for i in df.columns]  #creates column list based on dataframe columns#
        df_dict =  df.to_dict('records')                          #converts df to dictionary#
        message_output = 'Choose lookup method from dropdown, input coordinates, select output parameters, and click "Submit" button.\nID queries are limited to 20 entries, coordinate lookups must be done one at a time.'
        output_list = [column_list,df_dict,message_output]        #combines df_dict and column_list into output list#
        return output_list                                        #returns list of column names and data values#
    else:
        return [0,0,'Please limit each query to a maximum of 20 id numbers.']
        

if __name__ == '__main__':
    app.run_server()